# <div style="color:#fff;display:fill;border-radius:10px;background-color:#004F98;text-align:center;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%">Naïve Bayes Classifier</div>

## <span style='color:#2E8BC0'> 0| Import Libraries</span>

In [17]:
import numpy as np
import pandas as pd


In [18]:
dataset = pd.read_csv("titanic.csv")
df = dataset.copy()
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [19]:
df.shape

(891, 12)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Name         891 non-null    object 
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
 11  Survived     891 non-null    int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [21]:
df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Survived
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208,0.383838
std,257.353842,0.836071,14.526497,1.102743,0.806057,49.693429,0.486592
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,446.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,668.500000,3.000000,38.000000,1.000000,0.000000,31.000000,1.000000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [22]:
df.isnull().sum()

PassengerId      0
Name             0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Survived         0
dtype: int64

## <span style='color:#2E8BC0'> 1| Data Preprocessing </span>

In [23]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [24]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [25]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [26]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [27]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [28]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [29]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

## <span style='color:#2E8BC0'> 2| Handling Missing Values </span>

In [34]:
inputs.Age[:10]

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
6    54.000000
7     2.000000
8    27.000000
9    14.000000
Name: Age, dtype: float64

In [35]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


## <span style='color:#2E8BC0'> 3| Splitting Data </span>

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

## <span style='color:#2E8BC0'> 4| Building Model </span>

In [38]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [39]:
model.fit(X_train,y_train)

GaussianNB()

In [40]:
model.score(X_test,y_test)

0.8022388059701493

In [41]:
X_test[0:10]

,Pclass,Age,Fare,female
870,3,26.000000,7.8958,0
413,2,29.699118,0.0000,0
407,2,3.000000,18.7500,0
83,1,28.000000,47.1000,0
323,2,22.000000,29.0000,1
133,2,29.000000,26.0000,1
42,3,29.699118,7.8958,0
186,3,29.699118,15.5000,1
284,1,29.699118,26.0000,0
564,3,29.699118,8.0500,1


In [42]:
y_test[0:10]

870    0
413    0
407    1
83     0
323    1
133    1
42     0
186    1
284    0
564    0
Name: Survived, dtype: int64

In [43]:
model.predict(X_test[0:10])

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 1], dtype=int64)

In [44]:
model.predict_proba(X_test[:10])

array([[0.96562664, 0.03437336],
       [0.92174872, 0.07825128],
       [0.88393319, 0.11606681],
       [0.65195616, 0.34804384],
       [0.24466349, 0.75533651],
       [0.27443801, 0.72556199],
       [0.96734302, 0.03265698],
       [0.47765913, 0.52234087],
       [0.76811368, 0.23188632],
       [0.46868042, 0.53131958]])

## <span style='color:#2E8BC0'> 5| Calculate the score using cross validation</span>

In [45]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.712     , 0.8       , 0.744     , 0.82258065, 0.7016129 ])